In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd

import os
import random

from sentence_transformers import SentenceTransformer

/home/amir/miniconda3/envs/nlp/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [14]:
# model = SentenceTransformer("all-MiniLM-L6-v2").to(device).eval()

# Multilingual models
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2").to(device).eval()
# model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2").to(device).eval()

/home/amir/miniconda3/envs/nlp/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
# Walk into directoies to find images and convert them to vectors
txt_paths = []
sentences = []
for root, dirs, files in os.walk('data/'):
    for file in files:
        if file.endswith((".txt")):
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read().replace('\n', '')
                sentences.append(text)


            txt_paths.append(file_path)
sentences = list(set(sentences))

In [16]:
embeddings = model.encode(sentences)
embeddings.shape

(6391, 384)

### Similarity Search

In [17]:
idx = random.randint(0, len(sentences))

query_sentence = sentences[idx]
query = embeddings[idx]
query_sentence

'آمريكا از كاهش واردات نفت ايران از سوي تركيه استقبال كردتهران ر وزير امور خارجه آمريكا در خلال نشست موسوم به دوستان سوريه در بغداد از تصميم تركيه مبني بر كاهش واردات نفت خام از ايران به ميزان 20 درصد استقبال كرد به گزارش ايرنا به نقل از خبرگزاري فرانسه ؛ "هيلاري كلينتون " گفت : من از شنيدن تصميم تركيه مبني بر كاهش قابل توجه واردات نفت خام ازايران دلگرم شدم   ما از اين حركت استقبال مي كنيم  شركت ملي نفت تركيه به نام "توپراش " جمعه گذشته اعلام كرد كه واردات نفت از ايران را به ميزان 20 درصد كاهش داده است  اين شركت در بيانيه خود اعلام كرد: با در نظر گرفتن شرايط و بررسي هاي انجام شده ؛ اين تصميم گرفته شد كه واردات نفت خام از ايران 20درصد كاهش پيدا كند وزير نيروي تركيه پيش تر گفته بود كه كمبود نفت خود را با واردات نفت خام از ليبي جبران خواهد كرد تركيه كه يك سوم نفت مورد نياز خود را از ايران وارد مي كرد؛ به دنبال معافيت از دور جديد تحريم هاي آمريكا عليه ايران است  بر اساس قوانين جديدي كه در آمريكا تصويب شده اند؛ اين كشور موسسات مالي كشورهايي را كه با ايران معاملات نفتي دارند مشمول تحريم مي ك

In [18]:
similarities = model.similarity(query, embeddings)
similarities

tensor([[0.4737, 0.5752, 0.3745,  ..., 0.4985, 0.6848, 0.6593]])

In [19]:
values, indices = similarities.sort(descending=True)

In [20]:
values

tensor([[1.0000, 0.8345, 0.8232,  ..., 0.0579, 0.0536, 0.0207]])

In [21]:
indices

tensor([[3874, 4187, 2619,  ..., 4882, 2094, 4736]])

In [22]:
idx

3874

In [23]:
neighbors = [sentences[i] for i in indices[0][:10]]
df = pd.DataFrame(neighbors, columns=['Neigbors'])
df

,Neigbors
0,آمريكا از كاهش واردات نفت ايران از سوي تركيه ا...
1,ادعاي نرخ بالاي نفت و گاز ايران به مجلس تركيه...
2,خبرگزاري فرانسه :آمريكا لغو تحريم فروش نفت اي...
3,وزيرانرژي آمريكا: بازارجهاني قادر به جبران خل...
4,وال استريت ژورنال : كاهش بهاي نفت ايران را در...
5,فعالیت ۴۰ شرکت نفتی آمریکا در منطقه کردستان عر...
6,روزنامه آمريكايي : توافقنامه اخير بغداد -تهرا...
7,وزير انرژي امريكا مي گويد عدم صدور نفت ايران ...
8,پايگاه خبري ترك ايران ادعاي تركيه را رد كردآن...
9,مناسبات ايران و تركيه از نگاه تلويزيون بي بي...


In [24]:
with pd.option_context('display.max_colwidth', None):
  display(df)

,Neigbors
0,"آمريكا از كاهش واردات نفت ايران از سوي تركيه استقبال كردتهران ر وزير امور خارجه آمريكا در خلال نشست موسوم به دوستان سوريه در بغداد از تصميم تركيه مبني بر كاهش واردات نفت خام از ايران به ميزان 20 درصد استقبال كرد به گزارش ايرنا به نقل از خبرگزاري فرانسه ؛ ""هيلاري كلينتون "" گفت : من از شنيدن تصميم تركيه مبني بر كاهش قابل توجه واردات نفت خام ازايران دلگرم شدم ما از اين حركت استقبال مي كنيم شركت ملي نفت تركيه به نام ""توپراش "" جمعه گذشته اعلام كرد كه واردات نفت از ايران را به ميزان 20 درصد كاهش داده است اين شركت در بيانيه خود اعلام كرد: با در نظر گرفتن شرايط و بررسي هاي انجام شده ؛ اين تصميم گرفته شد كه واردات نفت خام از ايران 20درصد كاهش پيدا كند وزير نيروي تركيه پيش تر گفته بود كه كمبود نفت خود را با واردات نفت خام از ليبي جبران خواهد كرد تركيه كه يك سوم نفت مورد نياز خود را از ايران وارد مي كرد؛ به دنبال معافيت از دور جديد تحريم هاي آمريكا عليه ايران است بر اساس قوانين جديدي كه در آمريكا تصويب شده اند؛ اين كشور موسسات مالي كشورهايي را كه با ايران معاملات نفتي دارند مشمول تحريم مي كند"
1,"ادعاي نرخ بالاي نفت و گاز ايران به مجلس تركيه كشيده شد"" سزگين تانريكولو "" معاون حزب مخالف جمهوري خواه خلق در سئوالي با ادعاي اينكه قيمت نفت و گاز وارداتي تركيه از ايران بالاست و دولت آنكارا با اين وجود به خريد نفت و گاز از ايران ادامه مي دهد - خواستار پاسخ وزير تجارت و گمركات تركيه شد به گزارش روز دوشنبه ايرنا - وي با ارائه اين سئوال به رييس مجلس تركيه خواستار پاسخگويي "" حياتي يازيجي اوغلو"" وزير تجارت و گمركات تركيه به اين مساله شد تانريكولو در سئوال خود همچنين اين موضوع را مطرح كرد كه آيا ادعاهاي مربوط به اينكه ايران به علت تحريم هاي بين المللي اعلام كرده بود كه براي فروش نفت و گاز به تركيه 15 تا 20 درصد تخفيف مي دهد صحت دارديا خير ؟ در صورت صحت - مبالغ تخفيف داده شده به چه فرد و يا افرادي پرداخت شده است ؟تانريكولو با طرح اين سووال كه آيا در پرداخت ها به ايران بابت واردات نفت و گاز مسووليتهاي بين المللي تركيه از نظر دور داشته شده يا خير؟ خواستار اعلام اسامي بانكهاي مورد استفاده در تجارت با ايران وميزان حجم تجارت با اين كشور از طريق اين بانك ها شده است وي همچنين خواستار پاسخگويي وزير تجارت و گمركات تركيه به مساله تجارت طلا بين تركيه و ايران و روشن شدن تمامي ابعاد آن : ادعاهاي مربوط به سوء استفاده در اين تجارت و استفاده دولت آنكارا از تجارت طلا بين ايران و تركيه براي كاهش تراز پرداختهاي تركيه شده است"
2,خبرگزاري فرانسه :آمريكا لغو تحريم فروش نفت ايران به 9 كشور را تمديد كردتهران - خبرگزاري فرانسه روز پنجشنبه اعلام كرد كه آمريكا معافيت تحريم فروش نفت ايران به 9 كشور از جمله چين و هند را به مدت 6 ماه ديگرتمديد كرد اين گزارش مي افزايد- واشنگتن مي گويد- اتخاذ سياست هاي اقتصادي دركشورهاي چين - هند- مالزي - كره جنوبي - سنگاپور- جنوب آفريقا- سريلانكا-تركيه و تايوان براي جلوگيري و حتي توقف واردات نفت خام از ايران بوده و مربوط به تحريم هاي اقتصادي آمريكا عليه ايران به دليل برنامه هاي هسته اي اين كشور نيست جان كري وزير امور خارجه آمريكا در مصاحبه اي در همين رابطه گفت كه لغو تحريم هاي نفتي ايران از 20 مارس 2012 شروع و سپس به مدت 180 روزتمديد شده است اين گزارش مي افزايد: هيلاري كلينتون وزير امور خارجه سابق آمريكانيز چنين اقداماتي را در ماه دسامبر سال گذشته آغاز كرد كه شامل 9كشور مي شد جان كري نيز همين اقدام را در ماه مارس سال جاري و با معافيت تحريم هاي اقتصادي براي 11 كشور عضو اتحاديه اروپا از جمله ژاپن - بلژيك - جمهوري چك - فرانسه - آلمان - يونان - ايتاليا- لهستان - هلند- اسپانيا و انگليس اعمال كرد در مجموع بيست كشور مذكور حجم خريد نفت خام ايران را به طور قابل توجهي كاهش داده و يا به طور كامل معاملات خود با ايران را متوقف كرده بودند با اين حال وزير خارجه آمريكا گفت كه واشنگتن و جامعه جهاني در كناريكديگر به منظور ادامه فشار عليه نظام ايران تا زماني كه به نگراني ها در باره برنامه هسته اي اش پاسخ دهد- ايستاده اند
3,"وزيرانرژي آمريكا: بازارجهاني قادر به جبران خلاء نفت ايران استتهران ر ""ارنست مونيز"" وزير انرژي آمريكا گفت - در صورت تشديد تحريم هاي غرب عليه جمهوري اسلامي - بازار جهاني نفت قادر است خلاء ناشي از كاهش بيش از پيش صادرات نفتي ايران را جبران كند مونيز روز يكشنبه در گفت وگو خبرگزاري رويترز در وين گفت - صادرات ايران - اكنون تاثير خاصي بر بازار جهاني ندارد و خلاء ناشي از كاهش آن با افزايش توليد آمريكا و عراق و نيز ظرفيت چشمگير ذخاير 